RNN and LSTM 

resources:

https://colah.github.io/posts/2015-08-Understanding-LSTMs/

https://stanford.edu/~shervine/teaching/cs-229/cheatsheet-deep-learning#nn


In [7]:
from sklearn import model_selection
import yfinance as yf
import numpy as np
import pandas as pd
import torch
from torch import nn 
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [8]:
ticker='TSLA'
df = yf.download(ticker, start='2024-01-01', end='2025-07-01')
df = df[['Close']]
df

/tmp/ipykernel_11343/3246371683.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start='2024-01-01', end='2025-07-01')
[*********************100%***********************]  1 of 1 completed


Price,Close
Ticker,TSLA
Date,
2024-01-02,248.419998
2024-01-03,238.449997
2024-01-04,237.929993
2024-01-05,237.490005
2024-01-08,240.449997
...,...
2025-06-24,340.470001
2025-06-25,327.549988


In [9]:
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df)
sequence_length = 30

def create_sequence(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length])
    return np.array(X), np.array(y)

In [10]:

#input_size es el numero de features del input
#hidden_state es el numero de features en el hidden state
#que coincide con el hidde de salida
rnn = nn.LSTM(input_size=3, hidden_size=12, num_layers=3, batch_first=True)

#el input tiene forma NxLxHin
#Hin is el input size
x = torch.randn(5, 32, 3)
x.shape
output, _  = rnn(x)
output.shape

torch.Size([5, 32, 12])

In [11]:
class StockDataset(Dataset):
    
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float)
        self.y = torch.tensor(y, dtype=torch.float)
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return len(self.y)         

In [12]:
X, y = create_sequence(scaled, sequence_length)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=.2, shuffle=False)
train_dataset = StockDataset(x_train, y_train)
test_dataset = StockDataset(x_test, y_test)

train_dataloader = DataLoader(train_dataset, batch_size=5, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=5, shuffle=True)

In [13]:
class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=1):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
    
    # input es de la forma NxLxHin
    # N es tamaño de batch 
    # L es tamaño de la secuencia
    # Hin es tamaño del input
    def forward(self, x):
        # x es Nx10x1
        # out es Nx10x64
        out, hidden = self.lstm(x)
        # out es Nx64
        out = out[:, -1, :]
        # out es Nx1
        out = self.fc(out)
        return out
        

In [14]:
model = LSTMModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 1000
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for x_batch, y_batch in train_dataloader:
        out = model(x_batch)
        loss = criterion(out, y_batch)
        #print(f' x is {x_batch} , shape is {x_batch.shape}')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if (epoch + 1) % 10 == 0:
        print(f"loss in epoch {epoch + 1}, loss {total_loss/len(train_dataloader):.5f}")    
    

loss in epoch 10, loss 0.00238
loss in epoch 20, loss 0.00194
loss in epoch 30, loss 0.00166
loss in epoch 40, loss 0.00141
loss in epoch 50, loss 0.00134
loss in epoch 60, loss 0.00122
loss in epoch 70, loss 0.00123
loss in epoch 80, loss 0.00119
loss in epoch 90, loss 0.00116
loss in epoch 100, loss 0.00125
loss in epoch 110, loss 0.00110
loss in epoch 120, loss 0.00112
loss in epoch 130, loss 0.00119
loss in epoch 140, loss 0.00113
loss in epoch 150, loss 0.00116
loss in epoch 160, loss 0.00120
loss in epoch 170, loss 0.00120
loss in epoch 180, loss 0.00114
loss in epoch 190, loss 0.00105
loss in epoch 200, loss 0.00108
loss in epoch 210, loss 0.00112
loss in epoch 220, loss 0.00107
loss in epoch 230, loss 0.00104
loss in epoch 240, loss 0.00103
loss in epoch 250, loss 0.00103
loss in epoch 260, loss 0.00099
loss in epoch 270, loss 0.00104
loss in epoch 280, loss 0.00102
loss in epoch 290, loss 0.00098
loss in epoch 300, loss 0.00102
loss in epoch 310, loss 0.00096
loss in epoch 320

In [15]:

d = df.iloc[-sequence_length:]
d = scaler.fit_transform(d)
x_test = torch.tensor(d, dtype=torch.float)
x_out = torch.unsqueeze(x_test, 0)
x_out.shape


torch.Size([1, 30, 1])

In [16]:
model.eval()

print(x.shape)
with torch.no_grad():
    prediction = model(x_out).numpy()
    pred_original = scaler.inverse_transform(prediction)
    print(f"predition value is {pred_original}")
    
#scaler.inverse_transform(x_test)

torch.Size([5, 32, 3])
predition value is [[332.1742]]


In [17]:
def reset_weights(model):
    for layer in model.children():
        if hasattr(layer, 'reset_paramters'):
            layer.reset_parameters()

reset_weights(model)

print("model parameter after reset:")

for name, param in model.named_parameters():
    print(f"{name} : {param.data.mean():.4f}")


model parameter after reset:
lstm.weight_ih_l0 : -0.1103
lstm.weight_hh_l0 : -0.0069
lstm.bias_ih_l0 : -0.0705
lstm.bias_hh_l0 : -0.0769
fc.weight : 0.0039
fc.bias : 0.1302
